# Object-oriented programming

### Goals for this lesson
1. Learn how to use classes to encapsulate data and functions in Python.
2. Build a class that implements a 2D diffusion model.
3. Understand why you might want to adopt an object-oriented approach in your own research code.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

### Why should you use classes? or, POP vs. OOP
#### The world of POP
Up until now, we have mostly stayed in the world of "procedure-oriented programming" (POP). You write a series of instructions for the computer - "procedures" - hopefully start to organize them into functions, and then call those functions on some data. This is a fine approach when we're dealing with short programs and small amounts of data, but isn't always the best strategy when confronted with more complex problems.

#### Exercise 1) POP strategies
For each exercise below, think through how you would solve the problem using array operations, for-loops, and functions. Write down a few lines of pseudocode or even just comments to map out a strategy to solve each problem. The point is not to get your code working perfectly, but to get you to engage with a few examples where the POP paradigm struggles.

__1a. Model comparison.__

Let's say we have 5 different models we want to compare, each with different diffusivity values [k1, k2, ..., k5]. We want to run each of these models, using our functions from before. They'll have the same initial_profile array, but we want to compare the results from each model run after the fact.

__1b. Swapping out processes.__

After running 1a, above, we realize that we actually want to plot each model's progress as we go along, let's say every 10 time steps or so. And further, we also want to include a new process, called advect_one_step, after we run diffusion in each time step. Modify your previous solution to include those new requirements. (Don't worry about writing a function called advect_one_step, just pretend it is already defined for you.)

__1c. Multiple initial profiles.__

Finally, we now have to do this for 3 initial profiles: [step, convex, planar]. We want to include the same processes for all of them, but for the step profile we need to also need to include a new function, fix_shock_condition after advection. 

*Reflect: take a look at your notes or the pseudocode for each of these exercises. How much code did you write? How much code did you repeat? If you had to return to this in a month, or pass it off to a collaborator, how difficult would it be to understand? What if you needed to do the same exact series of steps for a different case study? If you found a mistake, would you be able to fix it in one place, or would you need to modify several different blocks of code?*



#### The world of OOP
A different way we can organize our code is called "object-oriented programming" (OOP). The core idea in OOP is to link our data with the functions that operate on the data. In Python, we'll do this with classes. Note that this isn't the only approach - you can totally solve all the problems I mentioned above with clever implementations of functions. But, there are a few really nice benefits to OOP in the context of scientific modeling:
- We have to organize our code into logical chunks.
- That means we can build big programs by dividing them into small pieces.
- It's really easy for users (that includes you!) to figure out which functions get to modify a chunk of data.
- It's easier to return to code that we wrote a long time ago and understand what's happening.
- We won't have to copy+paste or repeat code.
- If we want to change something, we only have to change it in one place.

# Intro to classes
Using a class consists of two steps: first we make a blueprint that defines the object, then we make an instance of the object to use it. If you think about it, we actually do the same thing with functions. Here's an example:

In [2]:
# "def" means we are defining a function

def my_function(arg_1, arg_2):  # the function gets a name, "my_function"
    # and it takes two arguments, "arg_1" and "arg_2"
    result = arg_1 * arg_2  # we can make new variables inside the function

    return result  # and at the end we should return something

Wait, we just ran the code and it didn't do anything! That's because all we've done here is define the function. To use it, we would have to do something like:

In [3]:
# use the function
answer = my_function(100, 2)
print(answer)

200


Let's keep this in mind as we look at classes. First, instead of using "def" we'll define a class by calling it ... a class!

##### ***Classes are objects that hold data and the functions that will operate on the data.***


In [4]:
class MyModel:
    pass

Without the <tt>pass</tt> statement, this raises an error - similar to the error you would get if you tried to make a function that didn't do anything. Let's keep going. The first thing we need to do is tell Python how to deal with a brand-new instance of this object. We'll do that by defining a "method." 

##### ***Methods are functions that belong to a particular class.***

In [5]:
class MyModel:
    def __init__(self, a):
        self.a = a

Let's take a sec and look carefully at what's going on here. First, we have a weird function that's idented a level. This is a method, and we'll treat it exactly like a function, with two exceptions:
1. Indent methods (along with everything else in a class) one level inside the class definition.
2. The first argument to a method should be "self."

Note that unlike a lot of things in Python, you can't name "self" whatever you want. In this case, it's a reserved keyword that will **always** refer to an instance of the object.

Also, what's with the weird double-underscores on either side of <tt>\_\_init\_\_</tt>? and why call it <tt>init</tt> instead of something like <tt>initialize</tt>? This is another reserved phrase in Python. Functions that we define ourselves, like <tt>my_function</tt>, we can name whatever we want. But some functions in Python are built-in - that is, they are given reserved names, and Python will call them using those reserved names in certain situations.

For example, <tt>\_\_init\_\_</tt> gets called when we make a new instance of the class:


In [6]:
model = MyModel(100)

Notice that we don't need to specify self. It is implicitly filled in, so we just need to include the arguments after it (in this case, *a*). This will be true every time we call a method. 

Now, all of the code we added to the init method was called when we made an instance of MyModel. (Try it out! Add a print statement inside the init method.) In this case, we told Python that *self* - the instance of the model we just made - should have an *attribute* that is equal to the argument a.

##### ***Attributes are pieces of data that belong to an instance of a class.***

We can access attributes using the same "dot" syntax that we used to make them:

In [7]:
model.a

100

To make a new method, we'll just keep adding text to the class definition. We still need to specify *self* as the first argument to the method. We can use class attributes inside methods - in fact, that's one great reason to use classes! And, we can either have them return results, or store those results as new attributes.

In [8]:
class MyModel:
    def __init__(self, a):
        self.a = a

    def add(self, b):
        self.b = self.a + b

    def add_and_return(self, b):
        return self.a + b
model = MyModel(2)
model.add(2)
print(model.b)
print(model.add_and_return(2))

4
4


**Important note:** attributes can be anything you want: strings, numbers, dictionaries, arrays - even other classes!

# Building a class
In the rest of this lesson, we're going to take our diffusion model and implement it within a class. As we go along, hopefully you'll start to see how object-oriented programming can be a useful approach for organizing our ideas.

### Exercise 2)

In [9]:
# Make a blank template for your diffusion class.
# 2a) Name it something descriptive, like DiffusionModel.
# 2b) Define the __init__ method - what should it take as arguments?




Note that the process of building a class is already asking us to think a bit more carefully about our model. What data do we need to provide to our model? Should any of the arguments have default values?

If we do want to add a default value, we can do that using the following syntax:

In [10]:
class MyModel:
    def __init__(self, a = 100):
        self.a = a

model = MyModel()
print(model.a)

100


Note that we can always override the default value. For example:

In [11]:
class MyModel:
    def __init__(self, a = 100):
        self.a = a

model = MyModel(1)
print(model.a)

1


We could also include the default value as a *class attribute*:

In [12]:
class MyModel:
    g = 10

    def __init__(self, a = 100):
        self.a = a

model = MyModel()
model.g

10


This means that any instance of the class will have access to it, and we *really* don't want users to change the value. Note that they still can, however. (Try it!) A bonus is that class attributes can be accessed without needing an instance of the class (by calling MyModel.g, for example).  

In [13]:
MyModel.g

10

### Quick aside about documentation


Before we write too much more code, we're going to take a moment to *document* our class. This will help users understand what it does, and help us remember our ideas when we return to the code later. First, let's add a good docstring. This is text that can be automatically read by many programs, including the built-in <tt>help()</tt> function.

In [14]:
# 2c) Add a docstring to the class. It looks like this:
# class ExampleClass:
#     """This is a one-line description of the class.
#     
#     This is a more detailed description of the class.
#     
#     Attributes:
#         foo: description
#         bar: description
#     """


In [15]:
# Try it out! Use the help() function to see the docstring.


Next, let's add two more small details to the <tt>\_\_init\_\_</tt> method. First, we'll add a top-level docstring, using the same triple-quotes as above. Then, we'll also add *type hints* to the arguments and return values. This is extra, and often unnecessary, but I (and many others in CSDMS) like to use it to help us quickly see and understand what a function is asking for and returning. You could use external programs like *mypy* or *typeguard* to enforce these type hints, but Python will ignore them by default.

In [16]:
# 2d) Add a docstring to the __init__ method, like this:
# def __init__(self, ...):
#     """I'm a docstring!"""

# 2e) Add type hints to the __init__ method, like this:
# def __init__(self, foo: np.ndarray, bar: int, baz: float = 1.0):
#     """I'm a docstring!"""

# Note you can also use type hints for the return value, like this:
# def some_method(self, foo: int) -> np.ndarray:
#     return np.arange(foo)


### Back to building our class
Now, let's finish up the rest of the DiffusionModel class. At this point, you should have everything you need.

![](../media/draw-the-owl.jpg)

Just kidding. If you want, the following cells will walk through one way to approach this exercise. Remember that there are lots of possible approaches, however, so if you want to come up with your own strategy that's okay too!

Here's a reminder of the problem we're going to solve. 

Given a 2D array of values $u$, a source term $S$, and a diffusivity $k$, evolve the following PDE forward in time:
$$
\frac{\partial u}{\partial t} + \nabla\cdot q = S,
$$
where we model the flux $q$ using Fourier's law:
$$
q = -k\nabla u.
$$
As long as $k$ is constant in space, we can substitute and get:
$$
\frac{\partial u}{\partial t} - k\nabla^2 u = S.
$$

Since this is a lesson about building classes in Python, and not about solutions to parabolic PDE's, let's make a few simplifying assumptions.
1. Exclude the source term, such that S = 0.
2. Let k be constant in space.
3. Limit ourselves to a regular grid, so the spacing is uniform in both dimensions.

At this point, you should be able to revise your <tt>\_\_init\_\_</tt> method to include 3 things:
- a scalar field (what should the data type for this be?)
- the grid spacing (with a different value for x and y)
- a diffusivity parameter (remember that it is constant in space!)
If you have more arguments in your <tt>\_\_init\_\_</tt>, that's okay too!

### Exercise 3) Writing a solver
Now, let's think about what our model needs to *do*. I'll propose three *methods* here, but feel free to add your own or rework things in a way that makes sense to you. 

I would start with the following methods:
1. calc_stable_time_step()

For reference, the stable time step $\Delta t$ for 2D diffusion is given by:
$$
\Delta t = C \frac{dx\cdot dy}{k},
$$
where $C$ is some constant (to start out, try 0.1 ish for this problem), $dx$ and $dy$ are the grid spacing, and $k$ is the diffusivity. Think about which of these values should be class attributes and which should be arguments to the method itself.

2. plot_field()

This one is fairly straightforward, but it's important to include ways to monitor your models before/during/after they run. I recommend using <tt>plt.imshow()</tt> to plot 2D data. Take a look at the API using help() or the [official documentation](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html). We'll expand on this step later.

3. run_one_step()

Finally, let's write the actual solver. The first task here is to get the rate of change, $\frac{\partial u}{\partial t}$ (see above). Previously, we used <tt>np.diff</tt> or similar, but you can also use <tt>np.gradient</tt>. Now that we're working in two dimensions, we'll have to take first and second derivatives in both *x* and *y*, so you should end up calculating 4 different arrays. A helpful identity is:
$$
\nabla^2 f = \frac{\partial^2 f}{\partial x^2} + \frac{\partial^2 f}{\partial y^2}.
$$

Then, the second task is to update our solution field. A forward time step will work well here, as long as we keep our time step within the stable limit. Don't forget to multiply by *dt*!


### Exercise 4) Plotting our results
Lastly, we want to include a way to show off our model results! Make a new method that plots the model results. I recommend using <tt>plt.imshow</tt>, since we are modeling a problem in two dimensions. Here are a few things you should include:
- A colorbar, so we can see the magnitude of the scalar field.
- An option for users to pass their own colormap to the method.
- A title that includes the amount of time that has elapsed in the model. (This might require you to add code to some of the other methods.)

### Exercise 5) Test out your code
Modify the following code block, using the class you designed. I've given you a starting condition (plot it to make sure it looks reasonable!). Your objective is to run the starting condition for 1 hour and see what your diffusion model predicts.

In [17]:
# Generate a starting condition
x = np.linspace(-2, 2, 20)
y = np.linspace(-2, 2, 20)
X, Y = np.meshgrid(x, y)
field = 5.0 * np.exp(-4 * (X**2 + Y**2))

# Define model parameters
spacing = (1.0, 1.0)
diffusivity = 0.1

# 5a) Initialize your model
# ...

# 5b) Plot the initial condition
# ...

# 5c) Calculate the stable time step
# ...

# 5d) Run the model for 1 hour
# ...

# 5e) Plot the final result
# ...


# Answer key

<details>
    <summary><b>Click here to see a solution</b></summary>

```python
# This is an example of the finalized Diffusion2D class
class Diffusion2D:
    """Solves the 2D diffusion problem on a regular grid.
    
    Attributes:
        field (np.ndarray): the 2D array of values to diffuse.
        spacing (tuple): the grid spacing in (x, y) coordinates.
        diffusivity (float): Parameter controlling the rate of diffusion.

    Methods:
        calc_stable_time_step(cfl: float = 0.1) -> float: Given a CFL number, calculate the stable time step for the model.
        run_one_step(dt: float): Run one time step of dt seconds.
        plot_field(): Plot the current field.
    """

    def __init__(self, field: np.ndarray, spacing: tuple, diffusivity: float):
        """Initialize the model with a field of values and diffusivity parameter."""
        self.field = field
        self.spacing = spacing
        self.k = diffusivity
        self.time_elapsed = 0.0

    def calc_stable_time_step(self, cfl: float = 0.1) -> float:
        """Calculate the stable time step for the model."""
        return cfl * self.spacing[0] * self.spacing[1] / self.k

    def run_one_step(self, dt: float):
        """Run one time step of dt seconds."""
        gradient_x = np.gradient(self.field, self.spacing[0], axis = 1, edge_order = 1)
        gradient_y = np.gradient(self.field, self.spacing[1], axis = 0, edge_order = 1)
        rate_of_change_x = -self.k * np.gradient(gradient_x, self.spacing[0], axis = 1, edge_order = 1)
        rate_of_change_y = -self.k * np.gradient(gradient_y, self.spacing[1], axis = 0, edge_order = 1)
        self.field -= dt * (rate_of_change_x + rate_of_change_y)
        self.time_elapsed += dt

    def plot_field(self, cmap: str = 'viridis'):
        """Plot the current field."""
        plt.imshow(self.field, cmap = cmap)
        plt.title("Model field after {} seconds".format(self.time_elapsed))
        plt.colorbar()
        plt.show()
</details>

<details>
    <summary><b>Click here to see a solution</b></summary>

```python
x = np.linspace(-2, 2, 20)
y = np.linspace(-2, 2, 20)
X, Y = np.meshgrid(x, y)
field = 5.0 * np.exp(-4 * (X**2 + Y**2))
spacing = (1.0, 1.0)

model = Diffusion2D(field, spacing, diffusivity = 0.1)
print(model.calc_stable_time_step())
model.plot_field()

for i in range(60 * 60):
    model.run_one_step(dt = 1.0)

model.plot_field()

</details>